## 『本次練習內容』
#### 運用這幾天所學觀念搭建一個CNN分類器

## 『本次練習目的』
  #### 熟悉CNN分類器搭建步驟與原理
  #### 學員們可以嘗試不同搭法，如使用不同的Maxpooling層，用GlobalAveragePooling取代Flatten等等

In [1]:
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import BatchNormalization
from keras.datasets import cifar10
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder


Using TensorFlow backend.


In [2]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

print(x_train.shape) #(50000, 32, 32, 3)

## Normalize Data
def normalize(X_train,X_test):
        mean = np.mean(X_train,axis=(0,1,2,3))
        std = np.std(X_train, axis=(0, 1, 2, 3))
        X_train = (X_train-mean)/(std+1e-7)
        X_test = (X_test-mean)/(std+1e-7) 
        return X_train, X_test,mean,std
    
    
## Normalize Training and Testset    
x_train, x_test,mean_train,std_train = normalize(x_train, x_test) 

(50000, 32, 32, 3)


In [3]:
## OneHot Label 由(None, 1)-(None, 10)
## ex. label=2,變成[0,0,1,0,0,0,0,0,0,0]
one_hot=OneHotEncoder()
y_train=one_hot.fit_transform(y_train).toarray()
y_test=one_hot.transform(y_test).toarray()

C:\Users\06006647\AppData\Local\conda\conda\envs\tf-gpu\lib\site-packages\sklearn\preprocessing\_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


In [5]:
classifier=Sequential()

#卷積組合
classifier.add(Conv2D(filters=32, kernel_size=(5,5), padding='same', input_shape=(32, 32, 3), activation='relu'))
#32,3,3,input_shape=(32,32,3),activation='relu''
classifier.add(BatchNormalization())

'''自己決定MaxPooling2D放在哪裡'''
classifier.add(MaxPooling2D(pool_size=(2,2)))

#卷積組合
classifier.add(Conv2D(filters=64, kernel_size=(3, 3), padding='same', activation='relu'))
classifier.add(BatchNormalization())

#flatten
classifier.add(Flatten())

#FC
classifier.add(Dense(128,kernel_initializer ='uniform', activation = 'relu')) #output_dim=100,activation=relu

#輸出
classifier.add(Dense(10, kernel_initializer="uniform",activation='softmax'))
classifier.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 32, 32, 32)        2432      
_________________________________________________________________
batch_normalization_3 (Batch (None, 32, 32, 32)        128       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 16, 16, 32)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 16, 16, 64)        18496     
_________________________________________________________________
batch_normalization_4 (Batch (None, 16, 16, 64)        256       
_________________________________________________________________
flatten_2 (Flatten)          (None, 16384)             0         
_________________________________________________________________
dense_3 (Dense)              (None, 128)               2097280   
__________

In [7]:

#超過兩個就要選categorical_crossentrophy
classifier.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
classifier.fit(x_train,y_train,batch_size=100,epochs=10)

Epoch 1/10
50000/50000 [==============================] - 16s 327us/step - loss: 0.0520 - acc: 0.9834
Epoch 2/10
50000/50000 [==============================] - 16s 313us/step - loss: 0.0427 - acc: 0.9861
Epoch 3/10
50000/50000 [==============================] - 16s 313us/step - loss: 0.0465 - acc: 0.9858
Epoch 4/10
50000/50000 [==============================] - 16s 313us/step - loss: 0.0404 - acc: 0.9874
Epoch 5/10
50000/50000 [==============================] - 16s 312us/step - loss: 0.0318 - acc: 0.9900
Epoch 6/10
50000/50000 [==============================] - 16s 313us/step - loss: 0.0328 - acc: 0.9899
Epoch 7/10
50000/50000 [==============================] - 16s 313us/step - loss: 0.0362 - acc: 0.9891
Epoch 8/10
50000/50000 [==============================] - 16s 313us/step - loss: 0.0379 - acc: 0.9884
Epoch 9/10
50000/50000 [==============================] - 16s 313us/step - loss: 0.0345 - acc: 0.9893
Epoch 10/10
50000/50000 [==============================] - 16s 313us/step - loss: 

## 預測新圖片，輸入影像前處理要與訓練時相同
#### ((X-mean)/(std+1e-7) ):這裡的mean跟std是訓練集的
## 維度如下方示範

In [15]:
input_example=(np.zeros(shape=(1,32,32,3))-mean_train)/(std_train+1e-7) 
classifier.predict(input_example)

array([[8.7448711e-07, 4.1012154e-09, 2.0121347e-06, 3.7131447e-02,
        1.6715112e-05, 5.4309838e-11, 4.0401323e-05, 8.3432777e-04,
        9.6197373e-01, 4.1910923e-07]], dtype=float32)